# gopigo3 model & control

The final objective of this chapter are:
- Create a model of our gopigo3 robot 
- use rviz package to view the different topics and nodes
- create a model of the virtual environment
- use gazebo package to simulate the robot kinematics and dynamics
- locate the robot in our created environment
- create our firsts programs to control the robot movement with obtacle avoidance


The final model represents the real gopigo3 robot we will use in the laboratory

Interesting complementary information could be found:

https://brjapon.medium.com/learning-robotics-with-ros-made-easy-12197c918dab

http://wiki.ros.org/Robots/gopigo3

https://robots.ros.org/gopigo3/

https://github.com/ros-gopigo3/gopigo3

https://bitbucket.org/theconstructcore/two-wheeled-robot-motion-planning/src/master/


![Getting Starter](./Images/1_gopigo3_UB.png)

## 1. rUBot gopigo3 model generation

First of all, we have to create the "gopigo3_description" package containing the gopigo3 model:

In [ ]:
cd ~/rubot_gopigo_ws/src
catkin_create_pkg gopigo3_description rospy
cd ..
catkin_make

Then open the .bashrc file and verify the environment variables and source to the proper workspace:

source ~/rubot_gopigo_ws/devel/setup.bash

To create our robot model, we use URDF files (Unified Robot Description Format). URDF file is an XML format file for representing a robot model.(http://wiki.ros.org/urdf/Tutorials)

We have created 2 folders for model description:
- URDF: folder where different URDF models are located
- meshes: folder where 3D body models in stl format are located.

You can reduce the amount of code in a URDF file using Xacro package. With this package you can use costants, simple math and macros to create your robot model easier and compact.

The main parts of URDF model are:
- links: diferent bodies/plastic elements
- joints: connection between 2 links 
- sensors & actuators plugins (2D camera, LIDAR and DC motors)

The link definition contains:
- visual properties: the origin, geometry and material
- collision properties: the origin and geomnetry
- inertial properties: the origin, mass and inertia matrix

The joint definition contains:
- joint Type (fixed, continuous)
- parent and child frames
- origin frame
- rotation axis

In the case or wright wheel:

In [ ]:
<!-- Right Wheel -->
  <link name="right_wheel">
    <visual>
      <origin xyz="0 0 0" rpy="1.570795 0 0" />
      <geometry>
          <cylinder length="0.05" radius="0.1" />
      </geometry>
      <material name="orange"/>
    </visual>
    <collision>
      <origin xyz="0 0 0" rpy="1.570795 0 0" />
      <geometry>
          <cylinder length="0.05" radius="0.1" />
      </geometry>
    </collision>
    <inertial>
      <origin xyz="0 0 0" rpy="1.570795 0 0" />
      <mass value="0.15"/>
      <inertia ixx="0.01" ixy="0.0" ixz="0.0" iyy="0.005" iyz="0.0" izz="0.005"/>
    </inertial>
  </link>
  
  <!-- Right Wheel joint -->
  <joint name="joint_right_wheel" type="continuous">
    <parent link="base_link"/>
    <child link="right_wheel"/>
    <origin xyz="0 -0.30 0.025" rpy="0 0 0" /> 
    <axis xyz="0 1 0" />
  </joint>

The gopigo3 model includes different sensors and actuators:

Sensors:
- a two-dimensional camera: correspondas to RBPi camera
- a 360º LIDAR sensor: corresponds to the EAI YDLIDAR X4 (https://www.robotshop.com/es/es/escaner-laser-360-ydlidar-x4.html)

Actuator:
- Differential drive actuator: based on 2 DC motors with encoders to obtain the Odometry information

The full model contains also information about the sensor and actuator controllers using specific Gazebo plugins (http://gazebosim.org/tutorials?tut=ros_gzplugins#Tutorial:UsingGazebopluginswithROS). 

Gazebo plugins give your URDF models greater functionality and can tie in ROS messages and service calls for sensor output and motor input. 

These plugins can be referenced through a URDF file, and to insert them in the URDF file, you have to follow the sintax:
- for Differential drive actuator:

In [ ]:
<robot>
  ... robot description ...
  <gazebo>
    <plugin name="gazebo_ros_lds_lfcd_controller" filename="libgazebo_ros_laser.so">
        <topicName>scan</topicName>
        <frameName>base_scan</frameName>
    </plugin>
  </gazebo>
  ... robot description ...
</robot>

In this gazebo plugin, the kinematics of the robot configuration is defined:
- Forward kinematics: obtaining the robot POSE (odometry) with the robot wheel speeds information
- Inverse kinematics: obtaining the robot wheels speds to reach specific robot POSE (odometry)

The equations are sumarized below

![Getting Starter](./Images/1_Drive_kine.png)

- for sensor plugin:

In [ ]:
<robot>
  ... robot description ...
  <link name="sensor_link">
    ... link description ...
  </link>

  <gazebo reference="sensor_link">
    <sensor type="camera" name="camera1">
      ... sensor parameters ...
      <plugin name="camera_controller" filename="libgazebo_ros_camera.so">
        ... plugin parameters ..
      </plugin>
    </sensor>
  </gazebo>

</robot>

#### Generated models in URDF
The diferent model files we have created in urdf folder are:
- gopigo3_v1.urdf: includes only the geometrical description of gopigo robot.
- gopigo3_v2.urdf: includes sensors and actuator:
    - the 2D camera and LDS sensors
    - the differential drive actuator
- gopigo3_v3.urdf: includes sensors and actuator:
    - the 2D camera, LDS and LIDAR sensors
    - the differential drive actuator
- gopigo3_v4.gazebo: includes corrections in the proposed model:
    - base_link geometry and inertia
    - 2D camera real position
    - LIDAR scaling and orientation

We use a specific "display.launch" launch file where we specify the robot model we want to open in rviz with a configuration specified in "urdf.rviz":

In [ ]:
<?xml version="1.0"?>
<launch>
  <!-- set these parameters on Parameter Server -->
  <param name="robot_description" textfile="$(find gopigo3_description)/urdf/gopigo3_v4.urdf" />
  <!-- send fake joint values -->
  <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher">
    <param name="use_gui" value="False"/>
  </node>
  <!-- Combine joint values -->
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"/>
  <!-- Show in Rviz   -->
  <node name="rviz" pkg="rviz" type="rviz" args="-d $(find gopigo3_description)/rviz/urdf.rviz"/>
</launch>

Launch the ROS visualization tool to check that the model is properly built. 
RViz only represents the robot visual features. You have available all the options to check every aspect of the appearance of the model

In [ ]:
roslaunch gopigo3_description display.launch

![Getting Starter](./Images/1_gopigo_basic_rviz.png)

![Getting Starter](./Images/1_gopigo3_rviz.png)

### gopigo3 model improvements
Some modifications in the initial model "gopigo3_init.gazebo" have been made to:
- locate the LIDAR in the correct position
- locate the camera axis in front position to see properly

Special attention has to be done to the YDLIDAR sensor.

- YDLIDAR X4 is a 360-degree two-dimensional rangefinder (hereinafter referred to as X4) developed by YDLIDAR team. 
- Based on the principle of triangulation, it is equipped with related optics, electronics and algorithm design to achieve high-frequency and high-precision distance measurement. 
- The mechanical structure rotates 360 degrees to continuously output the angle information as well as the point cloud data of the scanning environment while ranging.
-10m Ranging distance (2cm absolute error)

![Getting Started](./Images/1_ydlidar.png)

It is important to note that:
- the number of points of real YDLidar is 720 (one each half degree)
- the number of points of simulated Lidar is 360 (one each degree)

### Exercise:

Modify the gopigo3.gazebo model to take into account:
- Adapt the base-link geometry and inertia to the real gopigo3 robot

![Getting Started](./Images/1_mides.png)

#### Using Xacro package

Xacro (short for XML Macros) helps in reducing the overall size of the URDF file and makes it easier to read and maintain. It also allows us to create modules and reutilize them to create repeated structures, such as several arms or legs. With this package you can use costants, simple math and macros to create your robot model easier and compact.

Usefull information:
- http://wiki.ros.org/urdf/Tutorials/Using%20Xacro%20to%20Clean%20Up%20a%20URDF%20File
- https://www.theconstructsim.com/exploring-ros-2-wheeled-robot-part-02-xacros/

We have a public gopigo3_description package (https://github.com/ros-gopigo3/gopigo3) where the robot model is defined in xacro format within the files:
- gopigo3.urdf.xacro
- gopigo3.gazebo.xacro
- common_properties.xacro

To display the xacro model a specific display_xacro.launch file is created

To make convertion from xacro files to urdf file fotmat, type:
- xacro gopigo3.urdf.xacro > gopigo3.urdf

You can display this new model in rviz

In [ ]:
roslaunch gopigo3_description display.launch

![Getting Started](./Images/1_gopigo_stl.png)

### Exercise:
Create and Verify in the final model taking care about:
- Activate all the sensors: in model.urdf.xacro verify if all the sensors are activated
    - Camera
    - Lidar
    - Distance sensor
    - IMU (usually IMU is not activated)
- Differential drive plugin: 
    - Take care about the maximum acceleration of wheels and torque. Usually we have to decrease this value to avoid slidings. We take acceleration 0.5 and torque 1
    - Be sure the Odom TF is published <publishOdomTF>true</publishOdomTF>
- YDLidar plugin:
    - By default there are 500 samples with 0.72deg resolution you have to change this to 360 samples with 1deg resolution

In [ ]:
<!-- Which sensors to include -->
<xacro:arg name="imu" default="true" />
<!--xacro:property name="imu" value="true"/-->
<xacro:arg name="distance"        default="true"/>
<xacro:arg name="pi_camera"       default="true"/>
<xacro:arg name="lds"             default="true"/>

In [ ]:
        <scan>
          <horizontal>
            <!--samples>360</samples>
            <resolution>1</resolution-->
            <!--samples>500</samples-->
            <!--resolution>0.72</resolution-->
            <samples>720</samples>
            <resolution>0.5</resolution>
            <min_angle>0.0</min_angle>
            <max_angle>6.28319</max_angle>
          </horizontal>
        </scan>

## 2. rUBot gopigo3 spawn in world environment

In robotics research, always before working with a real robot, we simulate the robot behaviour in a virtual environment close to the real one. The dynamic simulation of a robot, is a better approach to examining the actual behavior of the robot rather than just using software. Rigid body mechanics, including mass and inertia, friction, damping, motor controllers, sensor detection properties, noise signals, and every aspect of the robot and the environment that can be retained in a model with reasonable accuracy is much less expensive when replicated in a simulator than if you tried to do this with physical hardware.

Gazebo is an open source 3D robotics simulator and includes an ODE physics engine and OpenGL rendering, and supports code integration for closed-loop control in robot drives. This is sensor simulation and actuator control.

We will create a new spawn.launch file to spawn the robot in an empty world

In [ ]:
<launch>
 <!-- We resume the logic in gazebo_ros package empty_world.launch, -->
  <include file="$(find gazebo_ros)/launch/empty_world.launch"/>
 <!-- Spawn gopigo3 robot into Gazebo -->
    <!-- Robot URDF definition -->
    <arg name="model" default="gopigo3" />
    <param name="robot_description" textfile="$(find gopigo3_description)/urdf/gopigo3_stl.urdf"/>
  <node name="spawn_model" pkg="gazebo_ros" type="spawn_model" output="screen"
      args="-urdf -model gopigo3 -param robot_description"/>
</launch>

A very simple world "gopigo3.world" is created using gazebo:
- sudo gazebo
- save as to "worlds" folder

The robot could be spawn in this world using this new spawn_world.launch file

In [ ]:
<launch>
    <!-- Define the needed parameters -->
    <arg name="world" default="gopigo3.world"/> 
    <arg name="model" default="gopigo3.urdf" />
    <arg name="x_pos" default="0.5"/>
    <arg name="y_pos" default="0.5"/>
    <arg name="z_pos" default="0.0"/>
  
    <include file="$(find gazebo_ros)/launch/empty_world.launch">
        <arg name="world_name" value="$(find gopigo3_description)/worlds/$(arg world)"/>
    </include>
    <!-- Spawn gopigo3 robot into Gazebo -->
       <!-- Robot URDF definition -->
       <param name="robot_description" textfile="$(find gopigo3_description)/urdf/$(arg model)"/>
    <node name="spawn_model" pkg="gazebo_ros" type="spawn_model" output="screen"
        args="-urdf -model gopigo3 -param robot_description -x $(arg x_pos) -y $(arg y_pos) -z $(arg z_pos)"/>
   </launch>

In [ ]:
roslaunch gopigo3_description spawn_world.launch
roslaunch gopigo3_description display.launch

![Getting Started](./Images/1_gopigo3_spawn_rviz.png)

Carefully!:
- If there is an error "libcurl: (51) SSL: no alternative certificate subject name matches target host name ‘api.ignitionfuel.org’" then follow instructions in: https://varhowto.com/how-to-fix-libcurl-51-ssl-no-alternative-certificate-subject-name-matches-target-host-name-api-ignitionfuel-org-gazebo-ubuntu-ros-melodic/

You can see the nodes and topics generated using rqt_graph

![Getting Started](./Images/01_rubot_spawn_rqt.png)

In order to kill the previous Gazebo process, type:

killall gzserver && killall gzclient

or type ctrl+r and kill

### Design the Project world

Here we have first to design the project world, for exemple a maze from where our rUBot gopigo3 has to navigate autonomously.

There is a very useful and simple tool to design a proper world: Building editor" in gazebo.

Open gazebo as superuser:

In [ ]:
sudo gazebo

You can build your world using "Building Editor" in Edit menu

![Getting Started](./Images/1_BuildingWorld1_1.png)

You can save:
- the generated model in a model folder (without extension)

Close the Builder Editor, modify the model position and add other elements if needed. Save:
- the generated world (with extension .world) in the world folder.

Once you finish is better to close the terminal you have work as superuser


### Exercise 3:
Generate a proper world corresponding to the real maze we have in the laboratory.

Save this world as gopigo3_maze.world

### Spawn the gopigo3 robot in project world

Now, spawn the gopigo3 robot in our generated world. You have to create a "navigation.launch" file:

In [ ]:
roslaunch gopigo3_description navigation.launch

![Getting Started](./Images/1_maze1.png)

To control the robot with the Keyboard you have to install the "teleop-tools" package:

Perhaps is needed to setup your Keys again:
- curl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -

In [ ]:
sudo apt-get install ros-melodic-teleop-tools
or
sudo apt-get install ros-melodic-teleop-twist-keyboard

Then you will be able to control the robot with the Keyboard typing:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel
or
rosrun teleop_twist_keyboard teleop_twist_keyboard.py

## 3. gopigo3 navigation control in the new world environment

Once the world has been generated we will create a ROS Package "rubot_control" to perform the autonomous navigation 

In [ ]:
cd ~/rubot_gopigo_ws/src
catkin_create_pkg rubot_control rospy std_msgs sensor_msgs geometry_msgs nav_msgs
cd ..
catkin_make

![Getting Started](./Images/1_rubot_control.png)

We will create now different navigation python files in "src" folder:
- move1_gopigo.py: to define a rubot movement with linear and angular speed
- move2_gopigo_param.py: to perform the same operation using params
- move3_gopigo_distance.py: to specify a maximum distance

Specific launch files have been created to launch the nodes and python files created above:

In [ ]:
roslaunch rubot_control rubot_move1.launch

In [ ]:
roslaunch rubot_control rubot_move2.launch

In [ ]:
roslaunch rubot_control rubot_move3.launch

![Getting Started](./Images/1_rubot_move3.png)

## gopigo3 autonomous navigation and obstacle avoidance

In order to navigate autonomously and avoid obstacles, we have created diferent python files in "src" folder:
- rubot_LIDAR_Test.py: to test the LIDAR distance readings and angles
- rubot_autonomous_navigation1.py: to perform a simple algorithm for navigation with obstacle avoidance

we will create also a "launch" folder including the corresponding launch files:
- rubot_lidar_test.launch
- rubot_nav1.launch

#### 1. LIDAR test
We have created a world to test the rubot model. This world is based on a wall to verify that the LIDAR see the obstacle in the correct angle. We have to launch the "rubot_lidar_test.launch" file in the "rubot_control" package.

In [ ]:
#! /usr/bin/env python

import rospy
from sensor_msgs.msg import LaserScan

def callback(msg):
    print len(msg.ranges)
    # values at 0 degree
    print msg.ranges[0]
    # values at 90 degree
    print msg.ranges[90]
    # values at 180 degree
    print msg.ranges[180]
    # values at 270 degree
    print msg.ranges[270]
    # values at 360 degree
    print msg.ranges[359]

rospy.init_node('scan_values')
sub = rospy.Subscriber('/scan', LaserScan, callback)
rospy.spin()

In [ ]:
roslaunch rubot_control rubot_lidar_test.launch

![Getting Started](./Images/1_lidar_test.png)

### Exercise 4:
Create a LIDAR plugin corresponding to the one we are using: 720 sample points and 0,5deg resolution. You need to modify the gopigo3.urdf file

Generate a world with obstacles in 0deg, 90deg, 180deg, 270deg and 360deg at different distances and test the LIDAR.

Modify the lidar_test.py file to test this new lidar and verify it.

![Getting Started](./Images/1_lidar_test2.png)

#### 2. Autonomous navigation with obstacle avoidance
We will use now the created world to test the autonomous navigation with obstacle avoidance performance. 

We have to launch the "rubot_nav1.launch" file in the "rubot_control" package.

In [ ]:
roslaunch rubot_control rubot_self_nav.launch

Careful:
- we have included in launch file: gazebo spawn, rviz visualization and rubot_nav node execution 
- Verify in rviz you have to change the fixed frame to "odom" frame

![Getting Started](./Images/1_rubot_self_nav.png)

The algorithm description functionality is:
- "rubot_self_nav.py": The Python script makes the robot go forward. 
    - LIDAR is allways searching the closest distance and the angle
    - when this distance is lower than a threshold, the robot goes backward with angular speed in the oposite direction of the minimum distance angle.

### Exercise 5:

Create another rubot_self_nav1.py file to improve the navigation process of our gopigo3 rubot in the laboratory maze

### Exercise 6:

Create another rubot_self_nav1.py file to improve the navigation process of our gopigo3 rubot in the laboratory maze following the right wall and the left wall.

Follow the instructions to create the follow_wall.py python file: https://www.theconstructsim.com/wall-follower-algorithm/

To verufy, type:
- 
